<a href="https://colab.research.google.com/github/kimiaameri/GrammarEnriched/blob/main/GrammarEnrichedLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
#!pip install params_flow

# Preprocess Section

In [33]:
import os
import glob
import gc
import nltk
from nltk import pos_tag
import pickle
import numpy as np

In [34]:
Chunks=(('FENJJ',('JJ','JJR','JJS')),('FENMD',('MD')) ,('FENLS',('LS')),('FENNN',('NN','NNP','NNS','NNP$','NNPS')),('FENRB',('RB','RBR','RBS')), 
       ('FENPP',('PRP','PRP$','PP')),('FENVB',('VB','VBD','VBG','VBN','VBP','VBZ')),('FENWP',('WP','WP$')), 
       ('FENNP',('NP')), ('FENVP',('VP')),('FENPDT',('PDT')),('FENPOS',('POS')),('FENSYM',('SYM')),
       ('FENCC',('CC')),('FENCD',('CD')),('FENDT',('DT')),('FENEX',('EX')),('FENFW',('FW')),('FENIN',('IN')),
        ('FENTO',('TO')),('FENUH',('UH')),('FENWDT',('WDT')), ('FENWR',('WRB','WRBS')) )

In [ ]:
from transformers import BertModel,BertTokenizer #,BertPreTrainedModel,BertModel, BertConfig,TFBertEmbeddings,TFBertModel,TFBertMainLayer

bert = BertModel.from_pretrained('bert-base-uncased')
modifiedEmbeddings=bert.embeddings.word_embeddings.weight
modifiedEmbeddings=modifiedEmbeddings.cpu().detach().numpy()
modifiedPosiotions=bert.embeddings.position_embeddings.weight
modifiedPosiotions=modifiedPosiotions.cpu().detach().numpy()

In [36]:
Newtokenizer = BertTokenizer.from_pretrained('/content/drive/MyDrive/Thesis/uncased_L-12_H-768_A-12', 
                                          do_lower_case=True)

## Data prepration  Section

In [37]:
def MakingChunkEncode(configuration, row):
    Dict={}
    bert_tokens = []
    Bert_Tags = []
    Tokens = nltk.word_tokenize(row[0])
    Tags = pos_tag(Tokens)
    for j,orig_token in enumerate(Tokens):
      try:
          bert_tokens.extend(Newtokenizer.tokenize(orig_token))
          tags=Newtokenizer.tokenize(orig_token)
          for tag in tags:
              Bert_Tags.append((tag,Tags[j][1]))
      except:
            pass
    Grammar_FE=[j[0] for i in Bert_Tags for j in Chunks if i[1] in j[1]]
    Grammars_POS=' '.join(Grammar_FE)
    Grammars_POS=Grammars_POS+Tokens[-1]
    encoded_sent= Encode_data(row[0],configuration)
    encoded_grammar=Encode_data(Grammars_POS,configuration)
    Dict={'Sentence':row[0],'Grammar': Grammars_POS,'Encode_Sentence': encoded_sent,'Encode_Grammar':encoded_grammar}
    gc.collect()
    return(Dict)

def Encode_data(data,configuration):

  encoded_data = Newtokenizer(data,truncation=True,add_special_tokens=True,max_length=configuration.seq_length,
                                        pad_to_max_length=True,return_attention_mask = True,return_tensors='tf')
  input_ids = np.array(encoded_data['input_ids'])
  token_type_ids=np.array(encoded_data['token_type_ids'])
  attention_masks = np.array(encoded_data['attention_mask'])
  input_data=[input_ids,token_type_ids,attention_masks]
  matrix_data=np.vstack([input_data[0],input_data[1],input_data[2]])
  return(matrix_data)

# Tokenizer Section

In [38]:
import collections
import logging
import os
import unicodedata
from typing import List, Optional

from tokenizers import BertWordPieceTokenizer

from transformers.tokenization_utils import PreTrainedTokenizer, _is_control, _is_punctuation, _is_whitespace
from transformers.tokenization_utils_fast import PreTrainedTokenizerFast

logger = logging.getLogger(__name__)

VOCAB_FILES_NAMES = {"vocab_file": "vocab.txt"}
PRETRAINED_VOCAB_FILES_MAP = {
    "vocab_file": {
        "bert-base-uncased": "/content/drive/MyDrive/Thesis/uncased_L-12_H-768_A-12/vocab.txt",
    }
}

PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES = {
    "bert-base-uncased": 512,
    "bert-large-uncased": 512,
    "bert-base-cased": 512,
    "bert-large-cased": 512,
}

PRETRAINED_INIT_CONFIGURATION = {
    "bert-base-uncased": {"do_lower_case": True},
    "bert-large-uncased": {"do_lower_case": True},
    "bert-base-cased": {"do_lower_case": False},
    "bert-large-cased": {"do_lower_case": False},
}


In [39]:
def load_vocab(vocab_file):
    """Loads a vocabulary file into a dictionary."""
    vocab = collections.OrderedDict()
    with open(vocab_file, "r", encoding="utf-8") as reader:
        tokens = reader.readlines()
    for index, token in enumerate(tokens):
        token = token.rstrip("\n")
        vocab[token] = index
    return vocab

def whitespace_tokenize(text):
    """Runs basic whitespace cleaning and splitting on a piece of text."""
    text = text.strip()
    if not text:
        return []
    tokens = text.split()
    return tokens

In [166]:
class BertTokenizer(PreTrainedTokenizer):
   
    vocab_files_names = VOCAB_FILES_NAMES
    pretrained_vocab_files_map = PRETRAINED_VOCAB_FILES_MAP
    pretrained_init_configuration = PRETRAINED_INIT_CONFIGURATION
    max_model_input_sizes = PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES

    def __init__(
        self,
        vocab_file,
        do_lower_case=True,
        do_basic_tokenize=True,
        #do_grammar_tokenize=True,
        never_split=None,
        unk_token="[UNK]",
        sep_token="[SEP]",
        pad_token="[PAD]",
        cls_token="[CLS]",
        mask_token="[MASK]",
        tokenize_chinese_chars=True,
        **kwargs
    ):
        super().__init__(
            unk_token=unk_token,
            sep_token=sep_token,
            pad_token=pad_token,
            cls_token=cls_token,
            mask_token=mask_token,
            **kwargs,
        )

        if not os.path.isfile(vocab_file):
            raise ValueError(
                "Can't find a vocabulary file at path '{}'. To load the vocabulary from a Google pretrained "
                "model use `tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)`".format(vocab_file)
            )
        self.vocab = load_vocab(vocab_file)
        self.ids_to_tokens = collections.OrderedDict([(ids, tok) for tok, ids in self.vocab.items()])
        self.do_basic_tokenize = do_basic_tokenize
        if do_basic_tokenize:
            self.basic_tokenizer = BasicTokenizer(
                do_lower_case=do_lower_case, never_split=never_split, tokenize_chinese_chars=tokenize_chinese_chars
            )
        self.wordpiece_tokenizer = WordpieceTokenizer(vocab=self.vocab, unk_token=self.unk_token)

    @property
    def vocab_size(self):
        return len(self.vocab)

    def get_vocab(self):
        return dict(self.vocab, **self.added_tokens_encoder)

    def _tokenize(self, text):
        split_tokens = []
        if self.do_basic_tokenize:
            for token in self.basic_tokenizer.tokenize(text, never_split=self.all_special_tokens):

                # If the token is part of the never_split set
                if token in self.basic_tokenizer.never_split:
                    split_tokens.append(token)
                else:
                    split_tokens += self.wordpiece_tokenizer.tokenize(token)
        else:
            split_tokens = self.wordpiece_tokenizer.tokenize(text)
        return split_tokens

    def _convert_token_to_id(self, token):
        """ Converts a token (str) in an id using the vocab. """
        return self.vocab.get(token, self.vocab.get(self.unk_token))

    def _convert_id_to_token(self, index):
        """Converts an index (integer) in a token (str) using the vocab."""
        return self.ids_to_tokens.get(index, self.unk_token)

    def convert_tokens_to_string(self, tokens):
        """ Converts a sequence of tokens (string) in a single string. """
        out_string = " ".join(tokens).replace(" ##", "").strip()
        return out_string

    def build_inputs_with_special_tokens(
        self, token_ids_0: List[int], token_ids_1: Optional[List[int]] = None
    ) -> List[int]:
        """
        Build model inputs from a sequence or a pair of sequence for sequence classification tasks
        by concatenating and adding special tokens.
        A BERT sequence has the following format:

        - single sequence: ``[CLS] X [SEP]``
        - pair of sequences: ``[CLS] A [SEP] B [SEP]``

        Args:
            token_ids_0 (:obj:`List[int]`):
                List of IDs to which the special tokens will be added
            token_ids_1 (:obj:`List[int]`, `optional`, defaults to :obj:`None`):
                Optional second list of IDs for sequence pairs.

        Returns:
            :obj:`List[int]`: list of `input IDs <../glossary.html#input-ids>`__ with the appropriate special tokens.
        """
        if token_ids_1 is None:
            return [self.cls_token_id] + token_ids_0 + [self.sep_token_id]
        cls = [self.cls_token_id]
        sep = [self.sep_token_id]
        return cls + token_ids_0 + sep + token_ids_1 + sep

    def get_special_tokens_mask(
        self, token_ids_0: List[int], token_ids_1: Optional[List[int]] = None, already_has_special_tokens: bool = False
    ) -> List[int]:
        """
        Retrieves sequence ids from a token list that has no special tokens added. This method is called when adding
        special tokens using the tokenizer ``prepare_for_model`` method.

        Args:
            token_ids_0 (:obj:`List[int]`):
                List of ids.
            token_ids_1 (:obj:`List[int]`, `optional`, defaults to :obj:`None`):
                Optional second list of IDs for sequence pairs.
            already_has_special_tokens (:obj:`bool`, `optional`, defaults to :obj:`False`):
                Set to True if the token list is already formatted with special tokens for the model

        Returns:
            :obj:`List[int]`: A list of integers in the range [0, 1]: 1 for a special token, 0 for a sequence token.
        """

        if already_has_special_tokens:
            if token_ids_1 is not None:
                raise ValueError(
                    "You should not supply a second sequence if the provided sequence of "
                    "ids is already formated with special tokens for the model."
                )
            return list(map(lambda x: 1 if x in [self.sep_token_id, self.cls_token_id] else 0, token_ids_0))

        if token_ids_1 is not None:
            return [1] + ([0] * len(token_ids_0)) + [1] + ([0] * len(token_ids_1)) + [1]
        return [1] + ([0] * len(token_ids_0)) + [1]


    def create_token_type_ids_from_sequences(
        self, token_ids_0: List[int], token_ids_1: Optional[List[int]] = None
    ) -> List[int]:
        """
        Creates a mask from the two sequences passed to be used in a sequence-pair classification task.
        A BERT sequence pair mask has the following format:
        ::
            0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1
            | first sequence    | second sequence |
        if token_ids_1 is None, only returns the first portion of the mask (0's).
        Args:
            token_ids_0 (:obj:`List[int]`):
                List of ids.
            token_ids_1 (:obj:`List[int]`, `optional`, defaults to :obj:`None`):
                Optional second list of IDs for sequence pairs.
        Returns:
            :obj:`List[int]`: List of `token type IDs <../glossary.html#token-type-ids>`_ according to the given
            sequence(s).
        """
        sep = [self.sep_token_id]
        cls = [self.cls_token_id]
        if token_ids_1 is None:
            return len(cls + token_ids_0 + sep) * [0]
        return len(cls + token_ids_0 + sep) * [0] + len(token_ids_1 + sep) * [1]


    def save_vocabulary(self, vocab_path):
        """
        Save the sentencepiece vocabulary (copy original file) and special tokens file to a directory.

        Args:
            vocab_path (:obj:`str`):
                The directory in which to save the vocabulary.

        Returns:
            :obj:`Tuple(str)`: Paths to the files saved.
        """
        index = 0
        if os.path.isdir(vocab_path):
            vocab_file = os.path.join(vocab_path, VOCAB_FILES_NAMES["vocab_file"])
        else:
            vocab_file = vocab_path
        with open(vocab_file, "w", encoding="utf-8") as writer:
            for token, token_index in sorted(self.vocab.items(), key=lambda kv: kv[1]):
                if index != token_index:
                    logger.warning(
                        "Saving vocabulary to {}: vocabulary indices are not consecutive."
                        " Please check that the vocabulary is not corrupted!".format(vocab_file)
                    )
                    index = token_index
                writer.write(token + "\n")
                index += 1
        return (vocab_file,)


In [41]:
class BasicTokenizer(object):
    """Runs basic tokenization (punctuation splitting, lower casing, etc.)."""

    def __init__(self, do_lower_case=True, never_split=None, tokenize_chinese_chars=True):

        if never_split is None:
            never_split = []
        self.do_lower_case = do_lower_case
        self.never_split = set(never_split)
        self.tokenize_chinese_chars = tokenize_chinese_chars

    def tokenize(self, text, never_split=None):
        # union() returns a new set by concatenating the two sets.
        never_split = self.never_split.union(set(never_split)) if never_split else self.never_split

        # This was added on November 1st, 2018 for the multilingual and Chinese
        # models. This is also applied to the English models now, but it doesn't
        # matter since the English models were not trained on any Chinese data
        # and generally don't have any Chinese data in them (there are Chinese
        # characters in the vocabulary because Wikipedia does have some Chinese
        # words in the English Wikipedia.).
        if self.tokenize_chinese_chars:
            text = self._tokenize_chinese_chars(text)
        orig_tokens = whitespace_tokenize(text)
        split_tokens = []
        for token in orig_tokens:
            if self.do_lower_case and token not in never_split:
                token = token.lower()
                token = self._run_strip_accents(token)
            split_tokens.extend(self._run_split_on_punc(token, never_split))

        output_tokens = whitespace_tokenize(" ".join(split_tokens))
        return output_tokens

    def _run_strip_accents(self, text):
        """Strips accents from a piece of text."""
        text = unicodedata.normalize("NFD", text)
        output = []
        for char in text:
            cat = unicodedata.category(char)
            if cat == "Mn":
                continue
            output.append(char)
        return "".join(output)

    def _run_split_on_punc(self, text, never_split=None):
        """Splits punctuation on a piece of text."""
        if never_split is not None and text in never_split:
            return [text]
        chars = list(text)
        i = 0
        start_new_word = True
        output = []
        while i < len(chars):
            char = chars[i]
            if _is_punctuation(char):
                output.append([char])
                start_new_word = True
            else:
                if start_new_word:
                    output.append([])
                start_new_word = False
                output[-1].append(char)
            i += 1

        return ["".join(x) for x in output]

    def _tokenize_chinese_chars(self, text):
        """Adds whitespace around any CJK character."""
        output = []
        for char in text:
            cp = ord(char)
            if self._is_chinese_char(cp):
                output.append(" ")
                output.append(char)
                output.append(" ")
            else:
                output.append(char)
        return "".join(output)

    def _is_chinese_char(self, cp):
        """Checks whether CP is the codepoint of a CJK character."""
        # This defines a "chinese character" as anything in the CJK Unicode block:
        #   https://en.wikipedia.org/wiki/CJK_Unified_Ideographs_(Unicode_block)
        #
        # Note that the CJK Unicode block is NOT all Japanese and Korean characters,
        # despite its name. The modern Korean Hangul alphabet is a different block,
        # as is Japanese Hiragana and Katakana. Those alphabets are used to write
        # space-separated words, so they are not treated specially and handled
        # like the all of the other languages.
        if (
            (cp >= 0x4E00 and cp <= 0x9FFF)
            or (cp >= 0x3400 and cp <= 0x4DBF)  #
            or (cp >= 0x20000 and cp <= 0x2A6DF)  #
            or (cp >= 0x2A700 and cp <= 0x2B73F)  #
            or (cp >= 0x2B740 and cp <= 0x2B81F)  #
            or (cp >= 0x2B820 and cp <= 0x2CEAF)  #
            or (cp >= 0xF900 and cp <= 0xFAFF)
            or (cp >= 0x2F800 and cp <= 0x2FA1F)  #
        ):  #
            return True

        return False

    def _clean_text(self, text):
        """Performs invalid character removal and whitespace cleanup on text."""
        output = []
        for char in text:
            cp = ord(char)
            if cp == 0 or cp == 0xFFFD or _is_control(char):
                continue
            if _is_whitespace(char):
                output.append(" ")
            else:
                output.append(char)
        return "".join(output)


In [158]:
class WordpieceTokenizer(object):
    """Runs WordPiece tokenization."""

    def __init__(self, vocab, unk_token, max_input_chars_per_word=100):
        self.vocab = vocab
        self.unk_token = unk_token
        self.max_input_chars_per_word = max_input_chars_per_word

    def tokenize(self, text):
        output_tokens = []
        for token in whitespace_tokenize(text):
            chars = list(token)
            if len(chars) > self.max_input_chars_per_word:
                output_tokens.append(self.unk_token)
                continue

            is_bad = False
            start = 0
            sub_tokens = []
            while start < len(chars):
                end = len(chars)
                cur_substr = None
                while start < end:
                    substr = "".join(chars[start:end])
                    if start > 0:
                        substr = "##" + substr
                    if substr in self.vocab:
                        cur_substr = substr
                        break
                    end -= 1
                if cur_substr is None:
                    is_bad = True
                    break
                sub_tokens.append(cur_substr)
                start = end

            if is_bad:
                output_tokens.append(self.unk_token)
            else:
                output_tokens.extend(sub_tokens)
        return output_tokens

In [42]:
class BertTokenizerFast(PreTrainedTokenizerFast):
    vocab_files_names = VOCAB_FILES_NAMES
    pretrained_vocab_files_map = PRETRAINED_VOCAB_FILES_MAP
    pretrained_init_configuration = PRETRAINED_INIT_CONFIGURATION
    max_model_input_sizes = PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES

    def __init__(
        self,
        vocab_file,
        do_lower_case=True,
        unk_token="[UNK]",
        sep_token="[SEP]",
        pad_token="[PAD]",
        cls_token="[CLS]",
        mask_token="[MASK]",
        clean_text=True,
        tokenize_chinese_chars=True,
        strip_accents=None,
        wordpieces_prefix="##",
        **kwargs
    ):
        super().__init__(
            BertWordPieceTokenizer(
                vocab_file=vocab_file,
                unk_token=unk_token,
                sep_token=sep_token,
                cls_token=cls_token,
                clean_text=clean_text,
                handle_chinese_chars=tokenize_chinese_chars,
                strip_accents=strip_accents,
                lowercase=do_lower_case,
                wordpieces_prefix=wordpieces_prefix,
            ),
            unk_token=unk_token,
            sep_token=sep_token,
            pad_token=pad_token,
            cls_token=cls_token,
            mask_token=mask_token,
            **kwargs,
        )

        self.do_lower_case = do_lower_case

    def build_inputs_with_special_tokens(self, token_ids_0, token_ids_1=None):
        output = [self.cls_token_id] + token_ids_0 + [self.sep_token_id]

        if token_ids_1:
            output += token_ids_1 + [self.sep_token_id]

        return output


    def create_token_type_ids_from_sequences(
        self, token_ids_0: List[int], token_ids_1: Optional[List[int]] = None
    ) -> List[int]:
        """
        Creates a mask from the two sequences passed to be used in a sequence-pair classification task.
        A BERT sequence pair mask has the following format:

        ::

            0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1
            | first sequence    | second sequence |

        if token_ids_1 is None, only returns the first portion of the mask (0's).

        Args:
            token_ids_0 (:obj:`List[int]`):
                List of ids.
            token_ids_1 (:obj:`List[int]`, `optional`, defaults to :obj:`None`):
                Optional second list of IDs for sequence pairs.

        Returns:
            :obj:`List[int]`: List of `token type IDs <../glossary.html#token-type-ids>`_ according to the given
            sequence(s).
        """
        sep = [self.sep_token_id]
        cls = [self.cls_token_id]
        if token_ids_1 is None:
            return len(cls + token_ids_0 + sep) * [0]
        return len(cls + token_ids_0 + sep) * [0] + len(token_ids_1 + sep) * [1]

# Config Section



In [43]:
from transformers import PretrainedConfig

In [67]:
class GrammarEnrichedLMConfig(PretrainedConfig):#Config

    model_type = "GrammarEnrichedLM"

    def __init__(
        self,
        vocab_size=30522,
        grammar_size=30522,
        hidden_size=768,
        num_hidden_layers=6,
        num_attention_heads=6,
        intermediate_size=3072,
        hidden_act="gelu",
        hidden_dropout_prob=0.1,
        attention_probs_dropout_prob=0.1,
        max_position_embeddings=512,
        type_vocab_size=2,
        initializer_range=0.02,
        layer_norm_eps=1e-12,
        pad_token_id=0,
        gradient_checkpointing=False,
        use_token_type           = True,
        use_position_embeddings  = True,
        use_grammar_embeddings  = True,
        position_embedding_type= "absolute",
        learning_rate = 0.001,
        weight_decay = 0.0001,
        batch_size = 256,
        num_epochs = 100,
        mlp_head_units = 2048,
        num_heads = 6,
        transformer_layers=6,
        seq_length=512,
        **kwargs
    ):
        super().__init__( **kwargs)
        self.vocab_size=vocab_size
        self.grammar_size=grammar_size
        self.hidden_size=hidden_size
        self.num_hidden_layers=num_hidden_layers
        self.num_attention_heads=num_attention_heads
        self.intermediate_size=intermediate_size
        self.hidden_act=hidden_act
        self.hidden_dropout_prob=hidden_dropout_prob
        self.attention_probs_dropout_prob=attention_probs_dropout_prob
        self.max_position_embeddings=max_position_embeddings
        self.type_vocab_size=type_vocab_size
        self.initializer_range=initializer_range
        self.layer_norm_eps=layer_norm_eps
        self.pad_token_id=pad_token_id
        self.gradient_checkpointing=gradient_checkpointing
        self.use_token_type           = use_token_type
        self.use_position_embeddings  = use_position_embeddings
        self.use_grammar_embeddings  = use_grammar_embeddings
        self.position_embedding_type= position_embedding_type
        self.learning_rate = learning_rate
        self.weight_decay = weight_decay
        self.batch_size = batch_size
        self.num_epochs = num_epochs
        self.mlp_head_units = mlp_head_units
        self.num_heads = num_heads
        self.transformer_layers=transformer_layers
        self.seq_length=seq_length


configuration = GrammarEnrichedLMConfig()

In [45]:
configuration.seq_length

512

# Activation Functions Section

In [46]:
import logging
import numpy as np
import tensorflow as tf
#from transformers import BertConfig
from transformers.file_utils import  MULTIPLE_CHOICE_DUMMY_INPUTS,MULTIPLE_CHOICE_DUMMY_INPUTS,add_code_sample_docstrings,add_start_docstrings
from transformers.modeling_tf_utils import TFMultipleChoiceLoss, TFPreTrainedModel,TFQuestionAnsweringLoss, TFSequenceClassificationLoss, TFTokenClassificationLoss, get_initializer, keras_serializable, shape_list
from transformers.tokenization_utils import BatchEncoding

def cast_bool_to_primitive(x, default_value=False):
  if x is None:
    return default_value
  return x

In [47]:
def gelu(x):
    """ Gaussian Error Linear Unit.
    Original Implementation of the gelu activation function in Google Bert repo when initially created.
        For information: OpenAI GPT's gelu is slightly different (and gives slightly different results):
        0.5 * x * (1 + torch.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * torch.pow(x, 3))))
        Also see https://arxiv.org/abs/1606.08415
    """
    cdf = 0.5 * (1.0 + tf.math.erf(x / tf.math.sqrt(2.0)))
    return x * cdf

def gelu_new(x):
    """Gaussian Error Linear Unit.
    This is a smoother version of the RELU.
    Original paper: https://arxiv.org/abs/1606.08415
    Args:
        x: float Tensor to perform activation.
    Returns:
        `x` with the GELU activation applied.
    """
    cdf = 0.5 * (1.0 + tf.tanh((np.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3)))))
    return x * cdf

def swish(x):
    return x * tf.sigmoid(x)

ACT2FN = {
    "gelu": tf.keras.layers.Activation(gelu),
    "relu": tf.keras.activations.relu,
    "swish": tf.keras.layers.Activation(swish),
    "gelu_new": tf.keras.layers.Activation(gelu_new),
}


# Embedding Section

In [201]:
class GrammarEnrichedLMEmbeddings(tf.keras.layers.Layer):
    """Construct the embeddings from word, grammar, position and token_type embeddings.
    """
    def __init__(self, configuration, **kwargs):
        super().__init__(**kwargs)
        self.vocab_size = configuration.vocab_size
        self.grammar_size=configuration.grammar_size
        self.hidden_size = configuration.hidden_size
        self.initializer_range = configuration.initializer_range

        self.position_embeddings = tf.keras.layers.Embedding(
            configuration.max_position_embeddings,
            configuration.hidden_size,
            #weights=[modifiedPosiotions], trainable=True,
            embeddings_initializer=get_initializer(self.initializer_range),
            name="position_embeddings",
        )
        self.token_type_embeddings = tf.keras.layers.Embedding(
            configuration.type_vocab_size,
            configuration.hidden_size,
            #weights=[modifiedEmbeddings], trainable=True,
            embeddings_initializer=get_initializer(self.initializer_range),
            name="token_type_embeddings",
        )

        self.token_grammar_embeddings = tf.keras.layers.Embedding(
            configuration.grammar_size,
            configuration.hidden_size,
            weights=[modifiedEmbeddings], trainable=True,
            #embeddings_initializer=get_initializer(self.initializer_range),
            name="token_grammar_embeddings",
        )


        # self.LayerNorm is not snake-cased to stick with TensorFlow model variable name and be able to load
        # any TensorFlow checkpoint file
        self.LayerNorm = tf.keras.layers.LayerNormalization(epsilon=configuration.layer_norm_eps, name="LayerNorm")
        self.dropout = tf.keras.layers.Dropout(configuration.hidden_dropout_prob)

    def build(self, input_shape):
        """Build shared word embedding layer """
        with tf.name_scope("word_embeddings"):
            # Create and initialize weights. The random normal initializer was chosen
            # arbitrarily, and works well.
            self.word_embeddings = self.add_weight(
                "weight",
                shape=[self.vocab_size, self.hidden_size],
                initializer=get_initializer(self.initializer_range),
            )
        super().build(input_shape)

    def call(self, inputs, mode="embedding", training=False):
        """Get token embeddings of inputs.
        Args:
            inputs: list of four int64 tensors with shape [batch_size, length]: (input_ids, grammar_ids, position_ids, token_type_ids)
            mode: string, a valid value is one of "embedding" and "linear".
        Returns:
            outputs: (1) If mode == "embedding", output embedding tensor, float32 with
                shape [batch_size, length, embedding_size]; (2) mode == "linear", output
                linear tensor, float32 with shape [batch_size, length, vocab_size].
        Raises:
            ValueError: if mode is not valid.

        Shared weights logic adapted from
            https://github.com/tensorflow/models/blob/a009f4fb9d2fc4949e32192a944688925ef78659/official/transformer/v2/embedding_layer.py#L24
        """
        if mode == "embedding":
            return self._embedding(inputs, training=training)
        elif mode == "linear":
            return self._linear(inputs)
        else:
            raise ValueError("mode {} is not valid.".format(mode))

    def _embedding(self, inputs, training=False):
        """Applies embedding based on inputs tensor."""
        
        input_ids, grammar_ids, position_ids, token_type_ids, inputs_embeds,grammar_embeds = inputs
        
        if input_ids is not None:
            input_shape = shape_list(input_ids)
        else:
            input_shape = shape_list(inputs_embeds)[:-1]

        seq_length = input_shape[1]
        if position_ids is None:
            position_ids = tf.range(seq_length, dtype=tf.int32)[tf.newaxis, :]
        if token_type_ids is None:
            token_type_ids = tf.fill(input_shape, 0)
        '''
        if we want to use BERT Tokenizer Embedding matrix for Grammars
        
        if grammar_ids is None:
            grammar_embeds = tf.gather(self.word_embeddings, grammar_ids)
        '''
        if inputs_embeds is None:
            inputs_embeds = tf.gather(self.word_embeddings, input_ids)

        position_embeddings = self.position_embeddings(position_ids)
        token_type_embeddings = self.token_type_embeddings(token_type_ids)        
        '''
        if we want to use keras Embedding layer with randomly initialized weights for Grammars
        '''        
        grammar_embeds = self.token_grammar_embeddings(grammar_ids)
        
        embeddings = inputs_embeds +grammar_embeds+ position_embeddings + token_type_embeddings
        embeddings = self.LayerNorm(embeddings)
        embeddings = self.dropout(embeddings, training=training)
        return embeddings

    def _linear(self, inputs):
        """Computes logits by running inputs through a linear layer.
            Args:
                inputs: A float32 tensor with shape [batch_size, length, hidden_size]
            Returns:
                float32 tensor with shape [batch_size, length, vocab_size].
        """
        batch_size = shape_list(inputs)[0]
        length = shape_list(inputs)[1]

        x = tf.reshape(inputs, [-1, self.hidden_size])
        logits = tf.matmul(x, self.word_embeddings, transpose_b=True)

        return tf.reshape(logits, [batch_size, length, self.vocab_size])


In [154]:
embeddings =GrammarEnrichedLMEmbeddings(configuration,name="embeddings")

# Attention Section

In [50]:
class GrammarEnrichedLMSelfAttention(tf.keras.layers.Layer):
    def __init__(self, configuration, **kwargs):
        super().__init__(**kwargs)
        if configuration.hidden_size % configuration.num_attention_heads != 0:
            raise ValueError(
                "The hidden size (%d) is not a multiple of the number of attention "
                "heads (%d)" % (configuration.hidden_size, configuration.num_attention_heads)
            )

        self.num_attention_heads = configuration.num_attention_heads
        assert configuration.hidden_size % configuration.num_attention_heads == 0
        self.attention_head_size = int(configuration.hidden_size / configuration.num_attention_heads)
        self.all_head_size = self.num_attention_heads * self.attention_head_size

        self.query = tf.keras.layers.Dense(
            self.all_head_size, kernel_initializer=get_initializer(configuration.initializer_range), name="query"
        )
        self.key = tf.keras.layers.Dense(
            self.all_head_size, kernel_initializer=get_initializer(configuration.initializer_range), name="key"
        )
        self.value = tf.keras.layers.Dense(
            self.all_head_size, kernel_initializer=get_initializer(configuration.initializer_range), name="value"
        )
        self.grammar = tf.keras.layers.Dense(
            self.all_head_size, kernel_initializer=get_initializer(configuration.initializer_range), name="grammar"
        )

        self.dropout = tf.keras.layers.Dropout(configuration.attention_probs_dropout_prob)

    def transpose_for_scores(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_attention_heads, self.attention_head_size))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs, training=False):
        hidden_states, attention_mask, head_mask, output_attentions = inputs

        batch_size = shape_list(hidden_states)[0]
        mixed_query_layer = self.query(hidden_states)
        mixed_key_layer = self.key(hidden_states)
        mixed_value_layer = self.value(hidden_states)
        mixed_grammar_layer = self.grammar(hidden_states)

        query_layer = self.transpose_for_scores(mixed_query_layer, batch_size)
        key_layer = self.transpose_for_scores(mixed_key_layer, batch_size)
        value_layer = self.transpose_for_scores(mixed_value_layer, batch_size)
        grammar_layer = self.transpose_for_scores(mixed_grammar_layer, batch_size)

        # Take the dot product between "query" and "key" to get the raw attention scores.
        attention_scores = tf.matmul(
            query_layer, key_layer, transpose_b=True
        )  # (batch size, num_heads, seq_len_q, seq_len_k)
        dk = tf.cast(shape_list(key_layer)[-1], tf.float32)  # scale attention_scores
        attention_scores = attention_scores / tf.math.sqrt(dk)

        if attention_mask is not None:
            # Apply the attention mask is (precomputed for all layers in TFBertModel call() function)
            attention_scores = attention_scores + attention_mask

        # Normalize the attention scores to probabilities.
        attention_probs = tf.nn.softmax(attention_scores, axis=-1)

        # This is actually dropping out entire tokens to attend to, which might
        # seem a bit unusual, but is taken from the original Transformer paper.
        attention_probs = self.dropout(attention_probs, training=training)

        # Mask heads if we want to
        if head_mask is not None:
            attention_probs = attention_probs * head_mask

        #context_layer = tf.matmul(attention_probs, value_layer) 
        #context_layer=tf.matmul(tensorflow.matmul(attention_probs, value_layer),grammar_layer,transpose_a=True )

        # ********************************** Adding another vector for grammar in attaention head
        context_layer=tf.matmul(attention_probs, value_layer)*grammar_layer
        context_layer = tf.transpose(context_layer, perm=[0, 2, 1, 3])
        context_layer = tf.reshape(
            context_layer, (batch_size, -1, self.all_head_size)
        )  # (batch_size, seq_len_q, all_head_size)

        outputs = (
            (context_layer, attention_probs) if cast_bool_to_primitive(output_attentions) is True else (context_layer,)
        )

        return outputs

In [51]:
class GrammarEnrichedLMSelfOutput(tf.keras.layers.Layer):
    def __init__(self, configuration, **kwargs):
        super().__init__(**kwargs)
        self.dense = tf.keras.layers.Dense(
            configuration.hidden_size, kernel_initializer=get_initializer(configuration.initializer_range), name="dense"
        )
        self.LayerNorm = tf.keras.layers.LayerNormalization(epsilon=configuration.layer_norm_eps, name="LayerNorm")
        self.dropout = tf.keras.layers.Dropout(configuration.hidden_dropout_prob)

    def call(self, inputs, training=False):
        hidden_states, input_tensor = inputs

        hidden_states = self.dense(hidden_states)
        hidden_states = self.dropout(hidden_states, training=training)
        hidden_states = self.LayerNorm(hidden_states + input_tensor)
        return hidden_states


In [52]:
class GrammarEnrichedLMAttention(tf.keras.layers.Layer):
    def __init__(self, configuration, **kwargs):
        super().__init__(**kwargs)
        self.self_attention = GrammarEnrichedLMSelfAttention(configuration, name="self")
        self.dense_output = GrammarEnrichedLMSelfOutput(configuration, name="output")

    def prune_heads(self, heads):
        raise NotImplementedError

    def call(self, inputs, training=False):
        input_tensor, attention_mask, head_mask, output_attentions = inputs

        self_outputs = self.self_attention(
            [input_tensor, attention_mask, head_mask, output_attentions], training=training
        )
        attention_output = self.dense_output([self_outputs[0], input_tensor], training=training)
        outputs = (attention_output,) + self_outputs[1:]  # add attentions if we output them
        return outputs

In [53]:
class GrammarEnrichedLMIntermediate(tf.keras.layers.Layer):
    def __init__(self, configuration, **kwargs):
        super().__init__(**kwargs)
        self.dense = tf.keras.layers.Dense(
            configuration.intermediate_size, kernel_initializer=get_initializer(configuration.initializer_range), name="dense"
        )
        if isinstance(configuration.hidden_act, str):
            self.intermediate_act_fn = ACT2FN[configuration.hidden_act]
        else:
            self.intermediate_act_fn = configuration.hidden_act

    def call(self, hidden_states):
        hidden_states = self.dense(hidden_states)
        hidden_states = self.intermediate_act_fn(hidden_states)
        return hidden_states

In [54]:
class GrammarEnrichedLMOutput(tf.keras.layers.Layer):
    def __init__(self, configuration, **kwargs):
        super().__init__(**kwargs)
        self.dense = tf.keras.layers.Dense(
            configuration.hidden_size, kernel_initializer=get_initializer(configuration.initializer_range), name="dense"
        )
        self.LayerNorm = tf.keras.layers.LayerNormalization(epsilon=configuration.layer_norm_eps, name="LayerNorm")
        self.dropout = tf.keras.layers.Dropout(configuration.hidden_dropout_prob)

    def call(self, inputs, training=False):
        hidden_states, input_tensor = inputs

        hidden_states = self.dense(hidden_states)
        hidden_states = self.dropout(hidden_states, training=training)
        hidden_states = self.LayerNorm(hidden_states + input_tensor)
        return hidden_states

In [55]:
def mlp(x, hidden_units, dropout_rate):
      hidden_unit=[hidden_units*2,hidden_units]
      for units in hidden_unit:
          x = tf.keras.layers.Dense(units, activation=tf.nn.gelu)(x)
          x = tf.keras.layers.Dropout(dropout_rate)(x)
      return x

# layers Section

In [56]:
class GrammarEnrichedLMLayer(tf.keras.layers.Layer):
  
    def __init__(self, configuration, **kwargs):
        super().__init__(**kwargs)
        self.attention = GrammarEnrichedLMAttention(configuration, name="attention")
        self.intermediate = GrammarEnrichedLMIntermediate(configuration, name="intermediate")
        self.grammarEnriched_output = GrammarEnrichedLMOutput(configuration, name="output")

    def call(self, inputs, training=False):
        hidden_states, attention_mask, head_mask, output_attentions = inputs

        attention_outputs = self.attention(
            [hidden_states, attention_mask, head_mask, output_attentions], training=training
        )
        attention_output = attention_outputs[0]
        intermediate_output = self.intermediate(attention_output)
        layer_output = self.grammarEnriched_output([intermediate_output, attention_output], training=training)
        outputs = (layer_output,) + attention_outputs[1:]  # add attentions if we output them
        return outputs

# Encoder Section

In [57]:
class GrammarEnrichedLMEncoder(tf.keras.layers.Layer):
    def __init__(self, configuration, **kwargs):
        super().__init__(**kwargs)
        self.layer = [GrammarEnrichedLMLayer(configuration, name="layer_._{}".format(i)) for i in range(configuration.num_hidden_layers)]

    def call(self, inputs, training=False):
        hidden_states, attention_mask, head_mask, output_attentions, output_hidden_states = inputs

        all_hidden_states = ()
        all_attentions = ()
        for i, layer_module in enumerate(self.layer):
            if cast_bool_to_primitive(output_hidden_states) is True:
                all_hidden_states = all_hidden_states + (hidden_states,)

            layer_outputs = layer_module(
                [hidden_states, attention_mask, head_mask[i], output_attentions], training=training
            )
            hidden_states = layer_outputs[0]

            if cast_bool_to_primitive(output_attentions) is True:
                all_attentions = all_attentions + (layer_outputs[1],)

        # Add last layer
        if cast_bool_to_primitive(output_hidden_states) is True:
            all_hidden_states = all_hidden_states + (hidden_states,)

        outputs = (hidden_states,)
        if cast_bool_to_primitive(output_hidden_states) is True:
            outputs = outputs + (all_hidden_states,)
        if cast_bool_to_primitive(output_attentions) is True:
            outputs = outputs + (all_attentions,)
        return outputs  # outputs, (hidden states), (attentions)


In [58]:
class GrammarEnrichedLMPooler(tf.keras.layers.Layer):
    def __init__(self, configuration, **kwargs):
        super().__init__(**kwargs)
        self.dense = tf.keras.layers.Dense(
            configuration.hidden_size,
            kernel_initializer=get_initializer(configuration.initializer_range),
            activation="tanh",
            name="dense",
        )

    def call(self, hidden_states):
        # We "pool" the model by simply taking the hidden state corresponding
        # to the first token.
        first_token_tensor = hidden_states[:, 0]
        pooled_output = self.dense(first_token_tensor)
        return pooled_output

In [59]:
class GrammarEnrichedLMHeadTransform(tf.keras.layers.Layer):
    def __init__(self, configuration, **kwargs):
        super().__init__(**kwargs)
        self.dense = tf.keras.layers.Dense(
            configuration.hidden_size, kernel_initializer=get_initializer(configuration.initializer_range), name="dense"
        )
        if isinstance(configuration.hidden_act, str):
            self.transform_act_fn = ACT2FN[configuration.hidden_act]
        else:
            self.transform_act_fn = configuration.hidden_act
        self.LayerNorm = tf.keras.layers.LayerNormalization(epsilon=configuration.layer_norm_eps, name="LayerNorm")

    def call(self, hidden_states):
        hidden_states = self.dense(hidden_states)
        hidden_states = self.transform_act_fn(hidden_states)
        hidden_states = self.LayerNorm(hidden_states)
        return hidden_states


In [60]:

class GrammarEnrichedLMPredictionHead(tf.keras.layers.Layer):
    def __init__(self, configuration, input_embeddings, **kwargs):
        super().__init__(**kwargs)
        self.vocab_size = configuration.vocab_size
        self.transform = GrammarEnrichedLMHeadTransform(configuration, name="transform")

        # The output weights are the same as the input embeddings, but there is
        # an output-only bias for each token.
        self.input_embeddings = input_embeddings

    def build(self, input_shape):
        self.bias = self.add_weight(shape=(self.vocab_size,), initializer="zeros", trainable=True, name="bias")
        super().build(input_shape)

    def call(self, hidden_states):
        hidden_states = self.transform(hidden_states)
        hidden_states = self.input_embeddings(hidden_states, mode="linear")
        hidden_states = hidden_states + self.bias
        return hidden_states

In [61]:
class GrammarEnrichedMLMHead(tf.keras.layers.Layer):
    def __init__(self, configuration, input_embeddings, **kwargs):
        super().__init__(**kwargs)
        self.predictions = GrammarEnrichedLMPredictionHead(configuration, input_embeddings, name="predictions")

    def call(self, sequence_output):
        prediction_scores = self.predictions(sequence_output)
        return prediction_scores

In [62]:
class GrammarEnrichedNSPHead(tf.keras.layers.Layer):
    def __init__(self, configuration, **kwargs):
        super().__init__(**kwargs)
        self.seq_relationship = tf.keras.layers.Dense(
            2, kernel_initializer=get_initializer(configuration.initializer_range), name="seq_relationship"
        )

    def call(self, pooled_output):
        seq_relationship_score = self.seq_relationship(pooled_output)
        return seq_relationship_score

# main layers Section

In [63]:
parameters= GrammarEnrichedLMConfig(configuration)

In [220]:
class GrammarEnrichedMainLayer(tf.keras.layers.Layer):
    config_class=configuration
    configuration= GrammarEnrichedLMConfig(configuration)

    def __init__(self, configuration, **kwargs):
        super().__init__(**kwargs)
        self.num_hidden_layers = configuration.num_hidden_layers
        self.initializer_range = configuration.initializer_range
        self.output_attentions = configuration.output_attentions
        self.output_hidden_states = configuration.output_hidden_states

        self.embeddings = GrammarEnrichedLMEmbeddings(configuration, name="embeddings")
        self.encoder = GrammarEnrichedLMEncoder(configuration, name="encoder")
        self.pooler = GrammarEnrichedLMPooler(configuration, name="pooler")

    def get_input_embeddings(self):
        return self.embeddings

    def set_input_embeddings(self, value):
        self.embeddings.word_embeddings = value
        self.embeddings.vocab_size = value.shape[0]

    def _prune_heads(self, heads_to_prune):
        """ Prunes heads of the model.
            heads_to_prune: dict of {layer_num: list of heads to prune in this layer}
            See base class PreTrainedModel
        """
        raise NotImplementedError

    def call(
        self,
        inputs,
        #grammar_ids,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        grammar_embeds=None,
        output_attentions=None,
        output_hidden_states=None,
        training=False,
    ):
        if isinstance(inputs, (tuple, list)):
            input_ids = inputs[0]
            attention_mask = inputs[1] if len(inputs) > 2 else attention_mask
            token_type_ids = inputs[2] if len(inputs) > 3 else token_type_ids
            grammar_ids = inputs[3]
            position_ids = inputs[4] if len(inputs) > 4 else position_ids
            head_mask = inputs[5] if len(inputs) > 5 else head_mask
            inputs_embeds = inputs[6] if len(inputs) > 6 else inputs_embeds
            grammar_embeds = inputs[7] if len(inputs) > 7 else grammar_embeds
            output_attentions = inputs[8] if len(inputs) > 8 else output_attentions
            output_hidden_states = inputs[9] if len(inputs) > 9 else output_hidden_states
            assert len(inputs) <= 9, "Too many inputs."
        elif isinstance(inputs, (dict, BatchEncoding)):
            input_ids = inputs.get("input_ids")
            grammar_ids = inputs.get("grammar_ids")
            attention_mask = inputs.get("attention_mask", attention_mask)
            token_type_ids = inputs.get("token_type_ids", token_type_ids)
            position_ids = inputs.get("position_ids", position_ids)
            head_mask = inputs.get("head_mask", head_mask)
            inputs_embeds = inputs.get("inputs_embeds", inputs_embeds)
            grammar_embeds=inputs.get("grammar_embeds", grammar_embeds)
            output_attentions = inputs.get("output_attentions", output_attentions)
            output_hidden_states = inputs.get("output_hidden_states", output_hidden_states)
            assert len(inputs) <= 9, "Too many inputs."
        else:
            input_ids = inputs[0]
            grammar_ids=inputs[1]

        output_attentions = output_attentions if output_attentions is not None else self.output_attentions
        output_hidden_states = output_hidden_states if output_hidden_states is not None else self.output_hidden_states

        if input_ids is not None and inputs_embeds is not None:
            raise ValueError("You cannot specify both input_ids and inputs_embeds at the same time")
        elif grammar_ids is not None and grammar_embeds is not None:
            raise ValueError("You cannot specify both grammar_ids and grammar_embeds at the same time")
        elif input_ids is not None:
            input_shape = shape_list(input_ids)
        elif inputs_embeds is not None:
            input_shape = shape_list(inputs_embeds)[:-1]
        else:
            raise ValueError("You have to specify either input_ids or inputs_embeds")

        if attention_mask is None:
            attention_mask = tf.fill(input_shape, 1)
        if token_type_ids is None:
            token_type_ids = tf.fill(input_shape, 0)

        # We create a 3D attention mask from a 2D tensor mask.
        # Sizes are [batch_size, 1, 1, to_seq_length]
        # So we can broadcast to [batch_size, num_heads, from_seq_length, to_seq_length]
        # this attention mask is more simple than the triangular masking of causal attention
        # used in OpenAI GPT, we just need to prepare the broadcast dimension here.
        extended_attention_mask = attention_mask[:, tf.newaxis, tf.newaxis, :]

        # Since attention_mask is 1.0 for positions we want to attend and 0.0 for
        # masked positions, this operation will create a tensor which is 0.0 for
        # positions we want to attend and -10000.0 for masked positions.
        # Since we are adding it to the raw scores before the softmax, this is
        # effectively the same as removing these entirely.

        extended_attention_mask = tf.cast(extended_attention_mask, tf.float32)
        extended_attention_mask = (1.0 - extended_attention_mask) * -10000.0

        # Prepare head mask if needed
        # 1.0 in head_mask indicate we keep the head
        # attention_probs has shape bsz x n_heads x N x N
        # input head_mask has shape [num_heads] or [num_hidden_layers x num_heads]
        # and head_mask is converted to shape [num_hidden_layers x batch x num_heads x seq_length x seq_length]
        if head_mask is not None:
            raise NotImplementedError
        else:
            head_mask = [None] * self.num_hidden_layers
            # head_mask = tf.constant([0] * self.num_hidden_layers)

        embedding_output = self.embeddings([input_ids,grammar_ids, position_ids, token_type_ids, inputs_embeds,grammar_embeds], training=training)
        encoder_outputs = self.encoder(
            [embedding_output, extended_attention_mask, head_mask, output_attentions, output_hidden_states],
            training=training,
        )

        sequence_output = encoder_outputs[0]
        pooled_output = self.pooler(sequence_output)

        outputs = (sequence_output, pooled_output,) + encoder_outputs[
            1:
        ]  # add hidden_states and attentions if they are here
        return outputs  # sequence_output, pooled_output, (hidden_states), (attentions)


# Build model Section

In [134]:
GrammarEnrichedMainLayer(configuration)

# Pretrained Section

In [135]:
from transformers import TFPreTrainedModel

In [136]:
class GrammarEnrichedPreTrainedModel(TFPreTrainedModel):
    config_class=configuration
    base_model_prefix = "GrammarEnrichedLM"

# GrammarEnrich Model Section

In [137]:
class GrammarEnrichedModel(GrammarEnrichedPreTrainedModel):
    def __init__(self, configuration, *inputs, **kwargs):
        super().__init__(configuration, *inputs, **kwargs)
        self.GrammarEnrichedLM = GrammarEnrichedMainLayer(configuration, name="GrammarEnrichedLM")


    def call(self, inputs, **kwargs):

        outputs = self.GrammarEnrichedLM(inputs, **kwargs)
        return outputs

In [228]:
mainmodel=GrammarEnrichedMainLayer(configuration, name="GrammarEnrichedLM")

# Pre Training Section

In [139]:
class GrammarEnrichedForPreTraining(GrammarEnrichedPreTrainedModel):
    def __init__(self, configuration, *inputs, **kwargs):
        super().__init__(configuration, *inputs, **kwargs)

        self.GrammarEnrichedLM = GrammarEnrichedMainLayer(configuration, name="GrammarEnrichedLM")
        self.nsp = GrammarEnrichedNSPHead(configuration, name="nsp___cls")
        self.mlm = GrammarEnrichedMLMHead(configuration, self.GrammarEnrichedLM.embeddings, name="mlm___cls")

    def get_output_embeddings(self):
        return self.GrammarEnrichedLM.embeddings

    def call(self, inputs, **kwargs):

        outputs = self.GrammarEnrichedLM(inputs, **kwargs)

        sequence_output, pooled_output = outputs[:2]
        prediction_scores = self.mlm(sequence_output, training=kwargs.get("training", False))
        seq_relationship_score = self.nsp(pooled_output)

        outputs = (prediction_scores, seq_relationship_score,) + outputs[
            2:
        ]  # add hidden states and attention if they are here

        return outputs  # prediction_scores, seq_relationship_score, (hidden_states), (attentions)


In [141]:
model=GrammarEnrichedMainLayer(configuration, name="GrammarEnrichedLM")
model

# MLM Section

In [140]:
class GrammarEnrichedForMaskedLM(GrammarEnrichedPreTrainedModel):
    def __init__(self, configuration, *inputs, **kwargs):
        super().__init__(configuration, *inputs, **kwargs)

        self.GrammarEnrichedLM = GrammarEnrichedMainLayer(configuration, name="GrammarEnrichedLM")
        self.mlm = GrammarEnrichedMLMHead(configuration, self.GrammarEnrichedLM.embeddings, name="mlm___cls")

    def get_output_embeddings(self):
        return self.GrammarEnrichedLM.embeddings

    def call(self, inputs, **kwargs):

        outputs = self.GrammarEnrichedLM(inputs, **kwargs)

        sequence_output = outputs[0]
        prediction_scores = self.mlm(sequence_output, training=kwargs.get("training", False))

        outputs = (prediction_scores,) + outputs[2:]  # Add hidden states and attention if they are here

        return outputs  # prediction_scores, (hidden_states), (attentions)


# NSP Section

In [142]:
class GrammarEnrichedLMForNextSentencePrediction(GrammarEnrichedPreTrainedModel):
    def __init__(self, configuration, *inputs, **kwargs):
        super().__init__(configuration, *inputs, **kwargs)

        self.GrammarEnrichedLM = GrammarEnrichedMainLayer(configuration, name="GrammarEnrichedLM")
        self.nsp = GrammarEnrichedNSPHead(configuration, name="nsp___cls")


    def call(self, inputs, **kwargs):
       
        outputs = self.GrammarEnrichedLM(inputs, **kwargs)

        pooled_output = outputs[1]
        seq_relationship_score = self.nsp(pooled_output)

        outputs = (seq_relationship_score,) + outputs[2:]  # add hidden states and attention if they are here

        return outputs  # seq_relationship_score, (hidden_states), (attentions)


# Fine Tune Example Section

## SequenceClassification Section

In [76]:
class GrammarEnrichedLMForSequenceClassification(GrammarEnrichedPreTrainedModel, TFSequenceClassificationLoss):
    def __init__(self, configuration, *inputs, **kwargs):
        super().__init__(configuration, *inputs, **kwargs)
        self.num_labels = configuration.num_labels

        self.GrammarEnrichedLM = GrammarEnrichedMainLayer(configuration, name="GrammarEnrichedLM")
        self.dropout = tf.keras.layers.Dropout(configuration.hidden_dropout_prob)
        self.classifier = tf.keras.layers.Dense(
            configuration.num_labels, kernel_initializer=get_initializer(configuration.initializer_range), name="classifier"
        )


    def call(
        self,
        inputs=None,
        grammar_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        grammar_embeds=None,
        output_attentions=None,
        output_hidden_states=None,
        labels=None,
        training=False,
    ):
 
        if isinstance(inputs, (tuple, list)):
            labels = inputs[11] if len(inputs) > 11 else labels
            if len(inputs) > 11:
                inputs = inputs[:11]
        elif isinstance(inputs, (dict, BatchEncoding)):
            labels = inputs.pop("labels", labels)

        outputs = self.GrammarEnrichedLM(
            inputs,
            grammar_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            grammar_embeds=grammar_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            training=training,
        )

        pooled_output = outputs[1]

        pooled_output = self.dropout(pooled_output, training=training)
        logits = self.classifier(pooled_output)

        outputs = (logits,) + outputs[2:]  # add hidden states and attention if they are here

        if labels is not None:
            loss = self.compute_loss(labels, logits)
            outputs = (loss,) + outputs

        return outputs  # (loss), logits, (hidden_states), (attentions)




## MultipleChoice Section

In [ ]:
class GrammarEnrichedForMultipleChoice(GrammarEnrichedPreTrainedModel, TFMultipleChoiceLoss):
    def __init__(self, configuration, *inputs, **kwargs):
        super().__init__(configuration, *inputs, **kwargs)

        self.GrammarEnrichedLM = GrammarEnrichedMainLayer(configuration, name="GrammarEnrichedLM")
        self.dropout = tf.keras.layers.Dropout(configuration.hidden_dropout_prob)
        self.classifier = tf.keras.layers.Dense(
            1, kernel_initializer=get_initializer(configuration.initializer_range), name="classifier"
        )

    def dummy_inputs(self):
        """ Dummy inputs to build the network.

        Returns:
            tf.Tensor with dummy inputs
        """
        return {"input_ids": tf.constant(MULTIPLE_CHOICE_DUMMY_INPUTS)}


    def call(
        self,
        inputs,
        grammar_ids,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        output_attentions=None,
        output_hidden_states=None,
        labels=None,
        training=False,
    ):
        if isinstance(inputs, (tuple, list)):
            input_ids = inputs[0]
            attention_mask = inputs[1] if len(inputs) > 1 else attention_mask
            token_type_ids = inputs[2] if len(inputs) > 2 else token_type_ids
            position_ids = inputs[3] if len(inputs) > 3 else position_ids
            head_mask = inputs[4] if len(inputs) > 4 else head_mask
            inputs_embeds = inputs[5] if len(inputs) > 5 else inputs_embeds
            output_attentions = inputs[6] if len(inputs) > 6 else output_attentions
            output_hidden_states = inputs[7] if len(inputs) > 7 else output_hidden_states
            labels = inputs[8] if len(inputs) > 8 else labels
            assert len(inputs) <= 9, "Too many inputs."
        elif isinstance(inputs, (dict, BatchEncoding)):
            input_ids = inputs.get("input_ids")
            attention_mask = inputs.get("attention_mask", attention_mask)
            token_type_ids = inputs.get("token_type_ids", token_type_ids)
            position_ids = inputs.get("position_ids", position_ids)
            head_mask = inputs.get("head_mask", head_mask)
            inputs_embeds = inputs.get("inputs_embeds", inputs_embeds)
            output_attentions = inputs.get("output_attentions", output_attentions)
            output_hidden_states = inputs.get("output_hidden_states", output_hidden_states)
            labels = inputs.get("labels", labels)
            assert len(inputs) <= 9, "Too many inputs."
        else:
            input_ids = inputs

        if input_ids is not None:
            num_choices = shape_list(input_ids)[1]
            seq_length = shape_list(input_ids)[2]
        else:
            num_choices = shape_list(inputs_embeds)[1]
            seq_length = shape_list(inputs_embeds)[2]

        flat_input_ids = tf.reshape(input_ids, (-1, seq_length)) if input_ids is not None else None
        flat_attention_mask = tf.reshape(attention_mask, (-1, seq_length)) if attention_mask is not None else None
        flat_token_type_ids = tf.reshape(token_type_ids, (-1, seq_length)) if token_type_ids is not None else None
        flat_position_ids = tf.reshape(position_ids, (-1, seq_length)) if position_ids is not None else None
        flat_inputs_embeds = (
            tf.reshape(inputs_embeds, (-1, seq_length, shape_list(inputs_embeds)[3]))
            if inputs_embeds is not None
            else None
        )

        flat_inputs = [
            flat_input_ids,
            flat_attention_mask,
            flat_token_type_ids,
            flat_position_ids,
            head_mask,
            flat_inputs_embeds,
            output_attentions,
            output_hidden_states,
        ]

        outputs = self.bert(flat_inputs, training=training)

        pooled_output = outputs[1]

        pooled_output = self.dropout(pooled_output, training=training)
        logits = self.classifier(pooled_output)
        reshaped_logits = tf.reshape(logits, (-1, num_choices))

        outputs = (reshaped_logits,) + outputs[2:]  # add hidden states and attention if they are here

        if labels is not None:
            loss = self.compute_loss(labels, reshaped_logits)
            outputs = (loss,) + outputs

        return outputs  # (loss), reshaped_logits, (hidden_states), (attentions)



## Token Classification Section

In [ ]:
class GrammarEnrichedLMForTokenClassification(GrammarEnrichedPreTrainedModel, TFTokenClassificationLoss):
    def __init__(self, configuration, *inputs, **kwargs):
        super().__init__(configuration, *inputs, **kwargs)
        self.num_labels = configuration.num_labels

        self.GrammarEnrichedLM = GrammarEnrichedMainLayer(configuration, name="GrammarEnrichedLM")
        self.dropout = tf.keras.layers.Dropout(configuration.hidden_dropout_prob)
        self.classifier = tf.keras.layers.Dense(
            configuration.num_labels, kernel_initializer=get_initializer(configuration.initializer_range), name="classifier"
        )

    def call(
        self,
        inputs=None,
        grammar_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        grammar_embeds=None,
        head_mask=None,
        inputs_embeds=None,
        output_attentions=None,
        output_hidden_states=None,
        labels=None,
        training=False,
    ):
       
        if isinstance(inputs, (tuple, list)):
            labels = inputs[8] if len(inputs) > 8 else labels
            if len(inputs) > 8:
                inputs = inputs[:8]
        elif isinstance(inputs, (dict, BatchEncoding)):
            labels = inputs.pop("labels", labels)

        outputs = self.bert(
            inputs,
            grammar_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            grammar_embeds=grammar_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            training=training,
        )

        sequence_output = outputs[0]

        sequence_output = self.dropout(sequence_output, training=training)
        logits = self.classifier(sequence_output)

        outputs = (logits,) + outputs[2:]  # add hidden states and attention if they are here

        if labels is not None:
            loss = self.compute_loss(labels, logits)
            outputs = (loss,) + outputs

        return outputs  # (loss), logits, (hidden_states), (attentions)




## QA Section

In [ ]:
class GrammarEnrichedLMForQuestionAnswering(GrammarEnrichedPreTrainedModel, TFQuestionAnsweringLoss):
    def __init__(self, configuration, *inputs, **kwargs):
        super().__init__(configuration, *inputs, **kwargs)
        self.num_labels = configuration.num_labels

        self.GrammarEnrichedLM = GrammarEnrichedMainLayer(configuration, name="GrammarEnrichedLM")
        self.qa_outputs = tf.keras.layers.Dense(
            configuration.num_labels, kernel_initializer=get_initializer(configuration.initializer_range), name="qa_outputs"
        )

    def call(
        self,
        inputs=None,
        grammar_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        grammar_embeds=None,
        output_attentions=None,
        output_hidden_states=None,
        start_positions=None,
        end_positions=None,
        training=False,
    ):

        if isinstance(inputs, (tuple, list)):
            start_positions = inputs[8] if len(inputs) > 8 else start_positions
            end_positions = inputs[9] if len(inputs) > 9 else end_positions
            if len(inputs) > 8:
                inputs = inputs[:8]
        elif isinstance(inputs, (dict, BatchEncoding)):
            start_positions = inputs.pop("start_positions", start_positions)
            end_positions = inputs.pop("end_positions", start_positions)

        outputs = self.bert(
            inputs,
            grammar_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            grammar_embed=grammar_embeds,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            training=training,
        )

        sequence_output = outputs[0]

        logits = self.qa_outputs(sequence_output)
        start_logits, end_logits = tf.split(logits, 2, axis=-1)
        start_logits = tf.squeeze(start_logits, axis=-1)
        end_logits = tf.squeeze(end_logits, axis=-1)

        outputs = (start_logits, end_logits,) + outputs[2:]

        if start_positions is not None and end_positions is not None:
            labels = {"start_position": start_positions}
            labels["end_position"] = end_positions
            loss = self.compute_loss(labels, outputs[:2])
            outputs = (loss,) + outputs

        return outputs  # (loss), start_logits, end_logits, (hidden_states), (attentions)


# Test Section

## MLM test Section

In [175]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [179]:
def PreProcessData(configuration, row):
    bert_tokens = []
    Bert_Tags = []
    Tokens = nltk.word_tokenize(row[0])
    Tags = pos_tag(Tokens)
    for j,orig_token in enumerate(Tokens):
      try:
          bert_tokens.extend(Newtokenizer.tokenize(orig_token))
          tags=Newtokenizer.tokenize(orig_token)
          for tag in tags:
              Bert_Tags.append((tag,Tags[j][1]))
      except:
            pass
    Grammar_FE=[j[0] for i in Bert_Tags for j in Chunks if i[1] in j[1]]
    Grammars_POS=' '.join(Grammar_FE)
    Grammars_POS=Grammars_POS+Tokens[-1]
    encoded_sent=Newtokenizer(row[0],
                    add_special_tokens=True, 
                    return_attention_mask=True, 
                    pad_to_max_length=True, 
                    max_length=10, 
                    truncation=True,
                    #padding=True,
                    return_tensors='tf',)
    encoded_grammar=Newtokenizer(Grammars_POS,
                    add_special_tokens=True, 
                    return_attention_mask=True, 
                    pad_to_max_length=True, 
                    max_length=10, 
                    truncation=True,
                    #padding=True,
                    return_tensors='tf',)
    encoded_sent['grammar_ids']=encoded_grammar.input_ids
    inputs=encoded_sent
    return(inputs)

In [143]:
input_ids = tf.constant(Newtokenizer.encode("Hello, my dog is cute", add_special_tokens=True))[None, :]  # Batch size 1
grammar_ids = tf.constant(Newtokenizer.encode("fennp, fennp fennp fenvp fenjj", add_special_tokens=True))[None, :]  # Batch size 1


In [225]:
inputs=PreProcessData(configuration,["Hello, my dog is cute"])


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2291: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [223]:
inputs

{'input_ids': <tf.Tensor: shape=(1, 10), dtype=int32, numpy=
array([[  101,  7592,  1010,  2026,  3899,  2003, 10140,   102,     0,
            0]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 10), dtype=int32, numpy=array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 10), dtype=int32, numpy=array([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0]], dtype=int32)>, 'grammar_ids': <tf.Tensor: shape=(1, 10), dtype=int32, numpy=
array([[  101,    54,   605,    54,   175,    21, 26869,   102,     0,
            0]], dtype=int32)>}

In [ ]:
model=GrammarEnrichedMainLayer(configuration, name="GrammarEnrichedLM")
model(inputs)

In [108]:
Newtokenizer.encode("Hello, my dog is cute", add_special_tokens=True)

[101, 7592, 1010, 2026, 3899, 2003, 10140, 102]

## NSP test Section

In [ ]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertForNextSentencePrediction

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = GrammarEnrichedLMForNextSentencePrediction.from_pretrained('GrammarEnrichedLM')

prompt = "In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced."
next_sentence = "The sky is blue due to the shorter wavelength of blue light."
encoding = tokenizer(prompt, next_sentence, return_tensors='tf')

logits = model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])[0]
assert logits[0][0] < logits[0][1] # the next sentence was random